In [1]:
import torch
import torch.nn as nn
import pandas as pd
import xgboost as xgb
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import grad as torch_grad
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import interp1d
from scipy.misc import derivative


In [52]:
train_x, train_y,raw_train_y, sliding_window_size,classification_label_size,trend_measure_lenth

def sliding_window(x, y, y_raw ,feature_window,label_window,trend_window):
    x_ = []
    y_ = []
    y_gan_pr = []
    y_gan_cl = []
    y_gan_tr = []
    backward_length = max(trend_window,label_window)
    for i in range(backward_length, x.shape[0]-label_window):
        tmp_x = x[i - feature_window: i, :]
        tmp_y = y[i]
        tmp_y_gan_pr = y[i - feature_window: i + 1]
        tmp_y_gan_cl = y_raw[i - label_window: i + label_window+1]
        tmp_y_gan_tr = y_raw[i - trend_window: i, :]
        x_.append(tmp_x)
        y_.append(tmp_y)
        y_gan_pr.append(tmp_y_gan_pr)
        y_gan_cl.append(tmp_y_gan_cl)
        y_gan_tr.append(tmp_y_gan_tr)
    x_ = torch.from_numpy(np.array(x_)).float()
    y_ = torch.from_numpy(np.array(y_)).float()
    # print(y_gan_cl.shape)
    # print(type(y_gan_cl))
    y_gan_pr = torch.from_numpy(np.array(y_gan_pr)).float()
    y_gan_cl = np.array(y_gan_cl)
    y_gan_cl = y_gan_cl.astype(np.float64)
    y_gan_cl = torch.from_numpy(np.array(y_gan_cl)).float()
    y_gan_tr = np.array(y_gan_tr)
    y_gan_tr = y_gan_tr.astype(np.float64)
    y_gan_tr = torch.from_numpy(np.array(y_gan_tr)).float()
    return x_, y_, y_gan_pr,y_gan_cl,y_gan_tr


In [53]:
# def trend_detection(data):
#     n = data.shape[1]
#     sets = data.shape[0]
#     mean_derivatives = np.zeros((sets,1))
    
#     data = data.numpy()

#     for i in range(sets):
#         # Select the i-th 20 set
#         x = np.arange(1,n+1)
#         x_fake = np.arange(1.1, n, 0.1)
#         y = data[i, :, 0]
#         # Simple interpolation of x and y    
#         f = interp1d(x, y)
        
#         # derivative of y with respect to x
#         df_dx = nd.Derivative(f, step=1e-6)(x_fake)
#         # Calculate the mean derivative for the i-th 20 set
#         average = np.average(df_dx)
#         mean_derivatives[i][0] = average
#     return  torch.from_num

def trend_detection(data):
    n = data.shape[1]
    sets = data.shape[0]
    mean_derivatives = np.zeros((sets, 1))

    for i in range(sets):
        x = np.arange(1, n + 1)
        x_fake = np.arange(1.1, n, 0.1)
        y = data[i, :, 0]

        # Linear interpolation of x and y
        f = np.interp(x_fake, x, y)

        # Calculate the derivatives
        df_dx = np.gradient(f, x_fake)

        # Calculate the mean derivative for the i-th set
        average = np.average(df_dx)
        mean_derivatives[i][0] = average

    return torch.from_numpy(mean_derivatives)   

In [54]:
class Generator(nn.Module):
    
    def __init__(self, input_size):
        super().__init__()

        # 3 GRU layers, input_size = features
        self.gru_1 = nn.GRU(input_size, 1024, batch_first=True)
        self.gru_2 = nn.GRU(1024, 512, batch_first = True)
        self.gru_3 = nn.GRU(512, 256, batch_first = True)
        # 3 Dense Layers
        self.linear_1 = nn.Linear(256, 128)
        self.linear_2 = nn.Linear(128, 64)
        self.linear_3 = nn.Linear(64, 1)

        self.dropout = nn.Dropout(0.2)
        self.tanh = nn.Tanh()


    def forward(self, x,use_cuda=0):
        h0 = torch.zeros(1, x.size(0), 1024) # initial hidden state for the 1st GRU Layer - (num of layers in the GRU, batch size, num of hidden units in the GRU)
        out_gru_1, _ = self.gru_1(x, h0)
        out_gru_1 = self.dropout(out_gru_1)

        h1 = torch.zeros(1, x.size(0), 512)
        out_gru_2, _ = self.gru_2(out_gru_1, h1)
        out_gru_2 = self.dropout(out_gru_2)

        h2 = torch.zeros(1, x.size(0), 256)
        out_gru_3, _ = self.gru_3(out_gru_2, h2)
        out_gru_3 = self.dropout(out_gru_3)

        out_dense_1 = self.linear_1(out_gru_3[:, -1, :])
        out_dense_2 = self.linear_2(out_dense_1)
        out_dense_3 = self.linear_3(out_dense_2)

        return out_dense_3,out_gru_3

In [56]:
asset = "BTC"
timestamp = "2021-04-08 13:50:00"

starting_index = 0
testing_duration = 22
sliding_window_size = 10
trend_measure_lenth = 12

training_duration = 4032
train_offset = 8640*4+4032
classification_label_size = 0

In [57]:
if asset == "BTC":
    df_init = pd.read_csv("dataset/initial/btc_init.csv")
    df = pd.read_csv("dataset/preprocessed/btc_pca.csv")
    wgan_model_url = "trained_models/wgan/wgan_btc.pth"
    xgb_model_url = "trained_models/bottom/BPC_WGAN_BTC.model"

elif asset == "ETH":
    df_init = pd.read_csv("dataset/initial/eth_init.csv")
    df = pd.read_csv("dataset/preprocessed/eth_pca.csv")
    wgan_model_url = "trained_models/wgan/wgan_eth.pth"
    xgb_model_url = "trained_models/bottom/BPC_WGAN_ETH.model"

elif asset == "LTC":
    df_init = pd.read_csv("dataset/initial/ltc_init.csv")
    df = pd.read_csv("dataset/preprocessed/ltc_pca.csv")
    wgan_model_url = "trained_models/wgan/wgan_ltc.pth"
    xgb_model_url = "trained_models/bottom/BPC_WGAN_LTC.model"

elif asset == "AAPL":
    df_init = pd.read_csv("dataset/initial/aapl_init.csv")
    df = pd.read_csv("dataset/preprocessed/aapl_pca.csv")
    wgan_model_url = "trained_models/wgan/wgan_aapl.pth"
    xgb_model_url = "trained_models/bottom/BPC_WGAN_AAPL.model"

elif asset == "TSLA":
    df_init = pd.read_csv("dataset/initial/tsla_init.csv")
    df = pd.read_csv("dataset/preprocessed/tsla_pca.csv")
    wgan_model_url = "trained_models/wgan/wgan_tsla.pth"
    xgb_model_url = "trained_models/bottom/BPC_WGAN_TSLA.model"

elif asset == "SBUX":
    df_init = pd.read_csv("dataset/initial/sbux_init.csv")
    df = pd.read_csv("dataset/preprocessed/sbux_pca.csv")
    wgan_model_url = "trained_models/wgan/wgan_sbux.pth"
    xgb_model_url = "trained_models/bottom/BPC_WGAN_SBUX.model"

In [58]:
df_init.columns

Index(['Unnamed: 0', 'Open_time', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Close_time', 'Quote_asset_volume', 'Number_of_trades',
       'Taker_buy_base_asset_volume', 'Taker_buy_quote_asset_volume',
       'Unknown', 'Datetime'],
      dtype='object')

In [59]:
index = df_init[df_init['Datetime'] == timestamp].index.item()

In [64]:
a = df_init.iloc[index:index]

Unnamed: 0                                    45670
Open_time                           1618010000000.0
Open                                       58154.87
High                                       58184.58
Low                                        58108.29
Close                                      58184.58
Volume                                   133.935882
Close_time                          1618010000000.0
Quote_asset_volume                      7787300.851
Number_of_trades                               4286
Taker_buy_base_asset_volume               68.109609
Taker_buy_quote_asset_volume              3960026.5
Unknown                                           0
Datetime                        2021-04-08 13:50:00
Name: 45670, dtype: object

In [81]:
labels_df = df["Close"]
features_df = df.drop(columns=["Close"])

train_x = features_df.iloc[train_offset:train_offset+training_duration]
train_y = labels_df.iloc[train_offset:train_offset+training_duration]

test_x = features_df.iloc[index-testing_duration:index+1]
test_y = labels_df.iloc[index-testing_duration:index+1]
real_y = df_init.iloc[index-10:index]

In [91]:
real_y = df_init.iloc[index-10:index]
real_y

,Unnamed: 0,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,Unknown,Datetime
45660,45660,1.618010e+12,58430.16,58430.17,58368.04,58376.27,59.445929,1.618010e+12,3471320.319,3260,27.469728,1604075.748,0,2021-04-08 13:00:00
45661,45661,1.618010e+12,58376.28,58385.42,58321.74,58361.24,97.938052,1.618010e+12,5714994.843,3765,49.064842,2863136.553,0,2021-04-08 13:05:00
45662,45662,1.618010e+12,58361.24,58361.24,58201.30,58264.10,138.791563,1.618010e+12,8087487.627,5370,66.534853,3876654.192,0,2021-04-08 13:10:00
45663,45663,1.618010e+12,58264.10,58294.88,58211.00,58232.12,83.108323,1.618010e+12,4841081.447,3493,50.694005,2952965.767,0,2021-04-08 13:15:00
45664,45664,1.618010e+12,58232.13,58272.15,58229.30,58260.49,84.873621,1.618010e+12,4943439.721,3312,52.779177,3074098.136,0,2021-04-08 13:20:00
45665,45665,1.618010e+12,58260.48,58262.80,58105.22,58178.00,131.702736,1.618010e+12,7660996.206,5248,47.561720,2766511.388,0,2021-04-08 13:25:00
45666,45666,1.618010e+12,58178.00,58225.93,58128.21,58200.01,104.519037,1.618010e+12,6081276.242,4277,57.195113,3328089.105,0,2021-04-08 13:30:00
45667,45667,1.618010e+12,58200.00,58229.06,58149.24,58190.72,100.349232,1.618010e+12,5839902.629,3925,58.730951,3417842.734,0,2021-04-08 13:35:00
45668,45668,1.618010e+12,58190.72,58278.53,58190.72,58201.78,70.695408,1.618010e+12,4117410.120,3204,40.784166,2375271.787,0,2021-04-08 13:40:00
45669,45669,1.618010e+12,58201.78,58245.09,58145.36,58154.87,99.175260,1.618010e+12,5771749.838,3815,48.699615,2834077.657,0,2021-04-08 13:45:00


In [82]:
test_y

45648    58322.58
45649    58331.66
45650    58405.63
45651    58410.82
45652    58350.99
45653    58401.54
45654    58413.98
45655    58440.67
45656    58494.99
45657    58468.68
45658    58423.86
45659    58430.17
45660    58376.27
45661    58361.24
45662    58264.10
45663    58232.12
45664    58260.49
45665    58178.00
45666    58200.01
45667    58190.72
45668    58201.78
45669    58154.87
45670    58184.58
Name: Close, dtype: float64

In [83]:
x_scaler = MinMaxScaler(feature_range = (0, 1))
y_scaler = MinMaxScaler(feature_range = (0, 1))
trend_scaler = MinMaxScaler(feature_range = (0, 1))

train_x = x_scaler.fit_transform(train_x)
test_x = x_scaler.transform(test_x)

raw_train_y = train_y.values.reshape(-1, 1)
raw_test_y = test_y.values.reshape(-1, 1)
train_y = y_scaler.fit_transform(train_y.values.reshape(-1, 1))
test_y = y_scaler.transform(test_y.values.reshape(-1, 1))

print(type(raw_train_y))
print(type(train_y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [84]:
# For price prediction model
train_x_slide, train_y_slide, train_y_gan,train_y_gan_cl,train_direction_slide = sliding_window(train_x, train_y,raw_train_y, sliding_window_size,classification_label_size,trend_measure_lenth)                
test_x_slide, test_y_slide, test_y_gan,test_y_gan_cl,test_direction_slide = sliding_window(test_x, test_y, raw_test_y ,sliding_window_size,classification_label_size,trend_measure_lenth)

train_trend_features = trend_detection(train_direction_slide)
test_trend_features = trend_detection(test_direction_slide)
train_trend_features = torch.tensor(trend_scaler.fit_transform(train_trend_features))
test_trend_features = torch.tensor(trend_scaler.transform(test_trend_features))
new_feature_tensor_train = train_trend_features.repeat(1,10)
new_feature_tensor_train = new_feature_tensor_train.unsqueeze(2)
new_feature_tensor_test = test_trend_features.repeat(1,10)
new_feature_tensor_test = new_feature_tensor_test.unsqueeze(2)

In [90]:
y_test_ac = y_scaler.inverse_transform(test_y_slide)
y_test_ac

array([[58376.26985849],
       [58361.23988995],
       [58264.09977984],
       [58232.12016359],
       [58260.48986965],
       [58177.99991339],
       [58200.0102297 ],
       [58190.71980673],
       [58201.78009927],
       [58154.87020192],
       [58184.5798407 ]])

In [86]:
def xgboost_test(x,xgmodel):
    out_gru_3_xg = x.reshape(x.shape[0], -1)
    pred = xgmodel.predict(out_gru_3_xg.detach().cpu().numpy())
    pred = torch.tensor(pred).reshape(pred.shape[0],1)
    return pred

In [87]:
modelG1 = torch.load(wgan_model_url, map_location=torch.device('cpu'))
state_dict = modelG1.state_dict()
modelG = Generator(train_x.shape[1])
modelG.load_state_dict(state_dict)
modelG.eval()

model_xgboost = xgb.XGBClassifier()
model_xgboost.load_model(xgb_model_url)

In [88]:
device = 'cpu'

price_prediction,test_input_features = modelG(test_x_slide.to(device))
test_input_features = torch.cat((test_input_features, new_feature_tensor_test), dim=2)
pred_direction_test = xgboost_test(test_input_features,model_xgboost)

y_test_true = y_scaler.inverse_transform(price_prediction.detach().numpy())

print(pred_direction_test)
print(y_test_true)

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]], dtype=torch.int32)
[[59408.27 ]
 [59394.46 ]
 [59384.746]
 [59369.25 ]
 [59360.89 ]
 [59377.734]
 [59368.547]
 [59338.953]
 [59357.582]
 [59392.71 ]
 [59342.094]]


In [102]:
pred_direction_test[-1]

tensor([0], dtype=torch.int32)

In [105]:
# Python Script

In [104]:
import torch
import torch.nn as nn
import pandas as pd
import xgboost as xgb
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import grad as torch_grad
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import interp1d
from scipy.misc import derivative

def sliding_window(x, y, y_raw ,feature_window,label_window,trend_window):
    x_ = []
    y_ = []
    y_gan_pr = []
    y_gan_cl = []
    y_gan_tr = []
    backward_length = max(trend_window,label_window)
    for i in range(backward_length, x.shape[0]-label_window):
        tmp_x = x[i - feature_window: i, :]
        tmp_y = y[i]
        tmp_y_gan_pr = y[i - feature_window: i + 1]
        tmp_y_gan_cl = y_raw[i - label_window: i + label_window+1]
        tmp_y_gan_tr = y_raw[i - trend_window: i, :]
        x_.append(tmp_x)
        y_.append(tmp_y)
        y_gan_pr.append(tmp_y_gan_pr)
        y_gan_cl.append(tmp_y_gan_cl)
        y_gan_tr.append(tmp_y_gan_tr)
    x_ = torch.from_numpy(np.array(x_)).float()
    y_ = torch.from_numpy(np.array(y_)).float()
    # print(y_gan_cl.shape)
    # print(type(y_gan_cl))
    y_gan_pr = torch.from_numpy(np.array(y_gan_pr)).float()
    y_gan_cl = np.array(y_gan_cl)
    y_gan_cl = y_gan_cl.astype(np.float64)
    y_gan_cl = torch.from_numpy(np.array(y_gan_cl)).float()
    y_gan_tr = np.array(y_gan_tr)
    y_gan_tr = y_gan_tr.astype(np.float64)
    y_gan_tr = torch.from_numpy(np.array(y_gan_tr)).float()
    return x_, y_, y_gan_pr,y_gan_cl,y_gan_tr

def trend_detection(data):
    n = data.shape[1]
    sets = data.shape[0]
    mean_derivatives = np.zeros((sets, 1))

    for i in range(sets):
        x = np.arange(1, n + 1)
        x_fake = np.arange(1.1, n, 0.1)
        y = data[i, :, 0]

        # Linear interpolation of x and y
        f = np.interp(x_fake, x, y)

        # Calculate the derivatives
        df_dx = np.gradient(f, x_fake)

        # Calculate the mean derivative for the i-th set
        average = np.average(df_dx)
        mean_derivatives[i][0] = average

    return torch.from_numpy(mean_derivatives)

class Generator(nn.Module):
    
    def __init__(self, input_size):
        super().__init__()

        # 3 GRU layers, input_size = features
        self.gru_1 = nn.GRU(input_size, 1024, batch_first=True)
        self.gru_2 = nn.GRU(1024, 512, batch_first = True)
        self.gru_3 = nn.GRU(512, 256, batch_first = True)
        # 3 Dense Layers
        self.linear_1 = nn.Linear(256, 128)
        self.linear_2 = nn.Linear(128, 64)
        self.linear_3 = nn.Linear(64, 1)

        self.dropout = nn.Dropout(0.2)
        self.tanh = nn.Tanh()


    def forward(self, x,use_cuda=0):
        h0 = torch.zeros(1, x.size(0), 1024) # initial hidden state for the 1st GRU Layer - (num of layers in the GRU, batch size, num of hidden units in the GRU)
        out_gru_1, _ = self.gru_1(x, h0)
        out_gru_1 = self.dropout(out_gru_1)

        h1 = torch.zeros(1, x.size(0), 512)
        out_gru_2, _ = self.gru_2(out_gru_1, h1)
        out_gru_2 = self.dropout(out_gru_2)

        h2 = torch.zeros(1, x.size(0), 256)
        out_gru_3, _ = self.gru_3(out_gru_2, h2)
        out_gru_3 = self.dropout(out_gru_3)

        out_dense_1 = self.linear_1(out_gru_3[:, -1, :])
        out_dense_2 = self.linear_2(out_dense_1)
        out_dense_3 = self.linear_3(out_dense_2)

        return out_dense_3,out_gru_3
    
def xgboost_test(x,xgmodel):
    out_gru_3_xg = x.reshape(x.shape[0], -1)
    pred = xgmodel.predict(out_gru_3_xg.detach().cpu().numpy())
    pred = torch.tensor(pred).reshape(pred.shape[0],1)
    return pred    

#Constants    
starting_index = 0
testing_duration = 22
sliding_window_size = 10
trend_measure_lenth = 12
training_duration = 4032
train_offset = 8640*4+4032
classification_label_size = 0
device = 'cpu'

def get_prediction(timestamp, asset):
    
    if asset == "BTC":
        df_init = pd.read_csv("dataset/initial/btc_init.csv")
        df = pd.read_csv("dataset/preprocessed/btc_pca.csv")
        wgan_model_url = "trained_models/wgan/wgan_btc.pth"
        xgb_model_url = "trained_models/bottom/BPC_WGAN_BTC.model"

    elif asset == "ETH":
        df_init = pd.read_csv("dataset/initial/eth_init.csv")
        df = pd.read_csv("dataset/preprocessed/eth_pca.csv")
        wgan_model_url = "trained_models/wgan/wgan_eth.pth"
        xgb_model_url = "trained_models/bottom/BPC_WGAN_ETH.model"

    elif asset == "LTC":
        df_init = pd.read_csv("dataset/initial/ltc_init.csv")
        df = pd.read_csv("dataset/preprocessed/ltc_pca.csv")
        wgan_model_url = "trained_models/wgan/wgan_ltc.pth"
        xgb_model_url = "trained_models/bottom/BPC_WGAN_LTC.model"

    elif asset == "AAPL":
        df_init = pd.read_csv("dataset/initial/aapl_init.csv")
        df = pd.read_csv("dataset/preprocessed/aapl_pca.csv")
        wgan_model_url = "trained_models/wgan/wgan_aapl.pth"
        xgb_model_url = "trained_models/bottom/BPC_WGAN_AAPL.model"

    elif asset == "TSLA":
        df_init = pd.read_csv("dataset/initial/tsla_init.csv")
        df = pd.read_csv("dataset/preprocessed/tsla_pca.csv")
        wgan_model_url = "trained_models/wgan/wgan_tsla.pth"
        xgb_model_url = "trained_models/bottom/BPC_WGAN_TSLA.model"

    elif asset == "SBUX":
        df_init = pd.read_csv("dataset/initial/sbux_init.csv")
        df = pd.read_csv("dataset/preprocessed/sbux_pca.csv")
        wgan_model_url = "trained_models/wgan/wgan_sbux.pth"
        xgb_model_url = "trained_models/bottom/BPC_WGAN_SBUX.model"
    
    # Index of the required timestep
    index = df_init[df_init['Datetime'] == timestamp].index.item()
    
    # Train test split
    labels_df = df["Close"]
    features_df = df.drop(columns=["Close"])
    train_x = features_df.iloc[train_offset:train_offset+training_duration]
    train_y = labels_df.iloc[train_offset:train_offset+training_duration]
    test_x = features_df.iloc[index-testing_duration:index+1]
    test_y = labels_df.iloc[index-testing_duration:index+1]
    
    # Scaling the Dataset 
    x_scaler = MinMaxScaler(feature_range = (0, 1))
    y_scaler = MinMaxScaler(feature_range = (0, 1))
    trend_scaler = MinMaxScaler(feature_range = (0, 1))
    train_x = x_scaler.fit_transform(train_x)
    test_x = x_scaler.transform(test_x)
    raw_train_y = train_y.values.reshape(-1, 1)
    raw_test_y = test_y.values.reshape(-1, 1)
    train_y = y_scaler.fit_transform(train_y.values.reshape(-1, 1))
    test_y = y_scaler.transform(test_y.values.reshape(-1, 1))
    
    # Input data preprocess
    train_x_slide, train_y_slide, train_y_gan,train_y_gan_cl,train_direction_slide = sliding_window(train_x, train_y,raw_train_y, sliding_window_size,classification_label_size,trend_measure_lenth)                
    test_x_slide, test_y_slide, test_y_gan,test_y_gan_cl,test_direction_slide = sliding_window(test_x, test_y, raw_test_y ,sliding_window_size,classification_label_size,trend_measure_lenth)
    train_trend_features = trend_detection(train_direction_slide)
    test_trend_features = trend_detection(test_direction_slide)
    train_trend_features = torch.tensor(trend_scaler.fit_transform(train_trend_features))
    test_trend_features = torch.tensor(trend_scaler.transform(test_trend_features))
    new_feature_tensor_train = train_trend_features.repeat(1,10)
    new_feature_tensor_train = new_feature_tensor_train.unsqueeze(2)
    new_feature_tensor_test = test_trend_features.repeat(1,10)
    new_feature_tensor_test = new_feature_tensor_test.unsqueeze(2)
    
    modelG1 = torch.load(wgan_model_url, map_location=torch.device('cpu'))
    state_dict = modelG1.state_dict()
    modelG2 = Generator(train_x.shape[1])
    modelG2.load_state_dict(state_dict)
    modelG2.eval()
    
    model_xgboost = xgb.XGBClassifier()
    model_xgboost.load_model(xgb_model_url)
    
    price_prediction,test_input_features = modelG2(test_x_slide)
    test_input_features = torch.cat((test_input_features, new_feature_tensor_test), dim=2)
    pred_direction_test = xgboost_test(test_input_features,model_xgboost)

    # predicted 11 steps
    y_test_pred = y_scaler.inverse_transform(price_prediction.detach().numpy())
    # Actuall 10 steps
    real_y = df_init.iloc[index-10:index]
    # Predicted bottom for timestep 11
    bottom_point = pred_direction_test[-1]
    print(y_test_pred)
    print()
    print(real_y)
    print()
    print(bottom_point)

asset = "BTC"
timestamp = "2021-04-08 13:50:00"    
get_prediction(timestamp,asset)    

[[59408.27 ]
 [59394.46 ]
 [59384.746]
 [59369.25 ]
 [59360.89 ]
 [59377.734]
 [59368.547]
 [59338.953]
 [59357.582]
 [59392.71 ]
 [59342.094]]

       Unnamed: 0     Open_time      Open      High       Low     Close  \
45660       45660  1.618010e+12  58430.16  58430.17  58368.04  58376.27   
45661       45661  1.618010e+12  58376.28  58385.42  58321.74  58361.24   
45662       45662  1.618010e+12  58361.24  58361.24  58201.30  58264.10   
45663       45663  1.618010e+12  58264.10  58294.88  58211.00  58232.12   
45664       45664  1.618010e+12  58232.13  58272.15  58229.30  58260.49   
45665       45665  1.618010e+12  58260.48  58262.80  58105.22  58178.00   
45666       45666  1.618010e+12  58178.00  58225.93  58128.21  58200.01   
45667       45667  1.618010e+12  58200.00  58229.06  58149.24  58190.72   
45668       45668  1.618010e+12  58190.72  58278.53  58190.72  58201.78   
45669       45669  1.618010e+12  58201.78  58245.09  58145.36  58154.87   

           Volume    Close_ti